In [1]:
!pip install torchmetrics
!pip install scipy
!pip install torch-fidelity

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 512 kB 11.0 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from google.colab import drive
drive.mount('/gdrive')
root = r'/gdrive/My Drive/CS492I/Final Term project/result/'

Mounted at /gdrive


In [3]:
from torchmetrics.image.fid import FrechetInceptionDistance
from torchmetrics.image.inception import InceptionScore
from torchvision.io import read_image
import os, torch
result_dir = r'/gdrive/MyDrive/CS492I/Final Term project/result'
gen_image_dir = r'reverse_vb_200_epoch/sample_imgs'
gen_image_path = os.path.join(result_dir, gen_image_dir)
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print("genated image path: ", gen_image_path)
print("use device: ", device)

genated image path:  /gdrive/MyDrive/CS492I/Final Term project/result/reverse_vb_200_epoch/sample_imgs
use device:  cuda:0


Load generated images

In [4]:
image_names = os.listdir(gen_image_path)
image_list = []

for f in image_names:
    image_list.append(read_image(os.path.join(gen_image_path, f)).cuda())
gen_images = torch.stack(image_list)

Load real images

In [5]:
from torchvision.datasets import CIFAR10
from torchvision.transforms import  Compose, ToTensor, Lambda

transform = Compose(
    [
        ToTensor(),
        Lambda(lambda x: x.to(device)),
    ]
)

cifar10_test = CIFAR10(root, train=False, transform = transform,download=True)

real_imgs = torch.from_numpy(cifar10_test.data).cuda()
real_imgs = real_imgs.permute(0,3,1,2)

Files already downloaded and verified


Load FID evaluator

In [6]:
fid = FrechetInceptionDistance(feature=2048).to(device)
inception_score = InceptionScore().to(device)
print("FID device: ", fid.device)
print("IS device: ", inception_score.device)

Downloading: "https://github.com/toshas/torch-fidelity/releases/download/v0.2.0/weights-inception-2015-12-05-6726825d.pth" to /root/.cache/torch/hub/checkpoints/weights-inception-2015-12-05-6726825d.pth


  0%|          | 0.00/91.2M [00:00<?, ?B/s]

/usr/local/lib/python3.8/dist-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `InceptionScore` will save all extracted features in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)


FID device:  cuda:0
IS device:  cuda:0


Update FID evaluator with data

In [7]:
batch_size = 128
num_batches = (gen_images.shape)[0] // batch_size 
gen_images[:128].shape

for i in range(num_batches):
    fid.update(gen_images[batch_size*i:batch_size*(i+1)], real=False)
    inception_score.update(gen_images[batch_size*i:batch_size*(i+1)])

if gen_images.shape[0] % 128 != 0:
    fid.update(gen_images[batch_size*num_batches:], real=False)    
    inception_score.update(gen_images[batch_size*num_batches:])

num_batches = real_imgs.shape[0] // batch_size
for i in range(num_batches):
    fid.update(real_imgs[batch_size*i:batch_size*(i+1)], real=True)
fid.update(real_imgs[batch_size*num_batches:], real=True)

Compute FID score, IS




In [8]:
fid_res = fid.compute()
is_res = inception_score.compute()
print(f"FID score: {fid_res.item()}")
print(f"Inception score: {is_res[0].item()}")



FID score: 65.36665344238281
Inception score: 5.285285472869873


In [9]:
from google.colab import runtime
runtime.unassign()